In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url_base = 'https://www.epey.com/akilli-telefonlar/{}/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Her sütun değerim için liste oluşturdum.
FiyatListe, IsimListe, PuanListe, DahiliDepolama, HatSayisi, RAM, EkranBoyutuListe, BataryaKapasitesi, BesG, DortBucukG, SuyaDayanaklilik, CPUFrekans, CPUCekirdek, EkranOran, KameraPixel, EkranPixel, ParmakIzi, SARDegeri,HizliSarj = ([] for _ in range(19))

for page_number in range(1,50):
    url = url_base.format(page_number)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    urunler = soup.find_all("ul", attrs={"class": "metin row"})
    for urun in urunler:
        urun_linkleri = urun.find_all("li", attrs={"class": "adi cell"})
        urun_fiyatlari = urun.find_all("li", attrs={"class": "fiyat cell"})
        urun_puanlari = urun.find_all("li", attrs={"class": "puan cell"})
        
        for i in urun_linkleri:
            link = i.find("div", attrs={"class": "detay cell"}).a.get("href")
            detay_soup = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
            ozellikler = detay_soup.find_all("div", attrs={"class": "tab table"})

            for j in ozellikler:
                telefon_isim = j.find("h2").text.split("(")[0].split("Temel")[0]
                ozeliik_detaylar = j.find_all("div", attrs={"class": "cell"})
                IsimListe.append(telefon_isim)
                
                # Aşağıdaki gibi isimlendirme yapmadığım zaman topladığım verileri birleştirme esnasında nA olan değerler problem oluşturuyordu.
                hatSayıisi = np.nan
                ekranBoyutu = np.nan
                dahiliDepolama = np.nan
                ram = np.nan
                batarya = np.nan
                hizliSarj = np.nan
                besG = np.nan
                dortBucukG = np.nan
                suyaDayanaklilik = np.nan
                frekans = np.nan
                cekirdek = np.nan
                ekranOran = np.nan
                kameraPixel = np.nan
                ekranPixel = np.nan
                parmakIzi = np.nan
                sarDegeri = np.nan
                hizliSarj = np.nan
                sarDegeri = np.nan
                
                print(str(page_number) + " " + telefon_isim)
                
                for k in ozeliik_detaylar:
                    
                    if(k.find("div", attrs={"class": "row row1"}).text=="Hat Sayısı:") :
                        hatSayıisi = k.find("div", attrs={"class": "row row2"}).text.strip()
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Ekran Boyutu:") :
                        ekranBoyutu = k.find("div", attrs={"class": "row row2"}).text.strip().split(" İnç")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Dahili Depolama:") :
                        dahiliDepolama = k.find("div", attrs={"class": "row row2"}).text.strip().split(" GB")[0].split(" TB")[0]    
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Bellek (RAM):") :
                        ram = k.find("div", attrs={"class": "row row2"}).text.strip().split(" GB")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Batarya Kapasitesi (Tipik):") :
                        batarya = k.find("div", attrs={"class": "row row2"}).text.strip().split(" mAh")[0]                
                    elif(k.find("div", attrs={"class": "row row1"}).text=="CPU Frekansı:") :
                        frekans = k.find("div", attrs={"class": "row row2"}).text.strip().split(" GHz")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="CPU Çekirdeği:") :
                        cekirdek = k.find("div", attrs={"class": "row row2"}).text.strip().split(" Çekirdek")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="SAR Değeri 10g (Baş):") :
                        sarDegeri = k.find("div", attrs={"class": "row row2"}).text.strip().split(" W/kg")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Ekran / Gövde Oranı:") :
                        ekranOran = k.find("div", attrs={"class": "row row2"}).text.strip().split(" %")[0]
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Kamera Çözünürlüğü:") :
                        kameraPixel = k.find("div", attrs={"class": "row row2"}).text.strip().split(" MP")[0]
                    
                    elif(k.find("div", attrs={"class": "row row1"}).text=="Ekran Çözünürlüğü:") :
                        if(k.find("div", attrs={"class": "row row2 font12"})):
                            ekranPixel = k.find("div", attrs={"class": "row row2 font12"}).text.strip().split(" (")[0].split("(")[0]
                        if(k.find("div", attrs={"class": "row row2"})):
                            ekranPixel = k.find("div", attrs={"class": "row row2"}).text.strip().split(" (")[0]
                        
                    elif(k.find("div", attrs={"class": "row row1"}).text=="5G:") :
                        if(k.find("div", attrs={"class": "row row2 degerYok"})) : 
                            besG = k.find("div", attrs={"class": "row row2 degerYok"}).text.strip()

                        if(k.find("div", attrs={"class": "row row2 degerVar"})) : 
                            besG = k.find("div", attrs={"class": "row row2 degerVar"}).text.strip()

                    elif(k.find("div", attrs={"class": "row row1"}).text=="Hızlı Şarj:") :
                        if(k.find("div", attrs={"class": "row row2 degerYok"})) : 
                            hizliSarj = k.find("div", attrs={"class": "row row2 degerYok"}).text.strip()

                        if(k.find("div", attrs={"class": "row row2 degerVar"})) : 
                            hizliSarj = k.find("div", attrs={"class": "row row2 degerVar"}).text.strip()

                    elif(k.find("div", attrs={"class": "row row1"}).text=="4.5G Desteği:") :
                        if(k.find("div", attrs={"class": "row row2 degerYok"})) : 
                            dortBucukG = k.find("div", attrs={"class": "row row2 degerYok"}).text.strip()

                        if(k.find("div", attrs={"class": "row row2 degerVar"})) : 
                            dortBucukG = k.find("div", attrs={"class": "row row2 degerVar"}).text.strip()

                    elif(k.find("div", attrs={"class": "row row1"}).text=="Suya Dayanıklılık:") :
                        if(k.find("div", attrs={"class": "row row2 degerYok"})) : 
                            suyaDayanaklilik = k.find("div", attrs={"class": "row row2 degerYok"}).text.strip()

                        if(k.find("div", attrs={"class": "row row2 degerVar"})) : 
                            suyaDayanaklilik = k.find("div", attrs={"class": "row row2 degerVar"}).text.strip()

                    elif(k.find("div", attrs={"class": "row row1"}).text=="Parmak izi Okuyucu:") :
                        if(k.find("div", attrs={"class": "row row2 degerYok"})) : 
                            parmakIzi = k.find("div", attrs={"class": "row row2 degerYok"}).text.strip()

                        if(k.find("div", attrs={"class": "row row2 degerVar"})) : 
                            parmakIzi = k.find("div", attrs={"class": "row row2 degerVar"}).text.strip()
                
                DahiliDepolama.append(dahiliDepolama)
                HatSayisi.append(hatSayıisi)
                RAM.append(ram)
                EkranBoyutuListe.append(ekranBoyutu)
                BataryaKapasitesi.append(batarya)
                BesG.append(besG)
                DortBucukG.append(dortBucukG)
                SuyaDayanaklilik.append(suyaDayanaklilik)
                ParmakIzi.append(parmakIzi)
                CPUFrekans.append(frekans)
                CPUCekirdek.append(cekirdek)
                SARDegeri.append(sarDegeri)
                EkranOran.append(ekranOran)
                EkranPixel.append(ekranPixel)
                KameraPixel.append(kameraPixel)
                HizliSarj.append(hizliSarj)
        
        for b in urun_fiyatlari:
            fiyatlar = b.find('a')
            if fiyatlar:
                # Her sayfanın ilk elemanı reklam olduğundan dolayı 'target' değeri '_blank' olanları kendi verime almadım.
                if "rel" in fiyatlar.attrs and fiyatlar["target"] == "_blank":
                    continue
                else:
                    fiyat = fiyatlar.text.strip()
                    fiyat_kismi = fiyat.split(" TL")[0]
                    fiyat_kismi = fiyat_kismi.replace(".", "").replace(",", ".")
            else:
                fiyat_kismi = None
            FiyatListe.append(fiyat_kismi)
            print(fiyat_kismi)
        for a in urun_puanlari:
            puan_div = a.find("div")
            puan = puan_div.get('data-text')
            PuanListe.append(puan)

1 Apple iPhone 15 Pro Max 
96315.03
1 Samsung Galaxy S23 Ultra 
63999.00
1 Apple iPhone 15 Pro 
81999.00
1 Xiaomi 13 Pro 
60969.00
1 Apple iPhone 14 Pro Max 
80702.05
1 OnePlus 11 
44989.99
1 Samsung Galaxy Z Fold5 
64999.00
1 Apple iPhone 14 Pro 
70949.00
1 Xiaomi 13T Pro 
36199.00
1 Samsung Galaxy S22 Ultra 
43218.00
1 Xiaomi 13 
47349.00
1 Samsung Galaxy Z Fold5 
57499.00
1 Samsung Galaxy S23+ 
37399.00
1 Samsung Galaxy Z Fold5 
53995.00
1 Huawei P60 Pro 
59999.00
1 Apple iPhone 13 Pro Max 
72421.99
1 Samsung Galaxy Z Fold4 
None
1 Samsung Galaxy S23 
29294.05
1 Huawei Mate X3 
74999.00
1 Huawei P60 Pro 
37999.00
1 Apple iPhone 15 Plus 
64497.60
1 Samsung Galaxy S21 Ultra 5G 
33600.00
1 Samsung Galaxy Z Fold4 
58799.02
1 Huawei Mate 50 Pro 
34999.00
1 Apple iPhone 13 Pro 
67619.02
1 Poco F5 Pro 
26148.00
1 Xiaomi 12 Pro 
41000.00
1 Samsung Galaxy S22+ 
31360.00
1 OnePlus 9 Pro 
None
1 Poco F5 Pro 
25799.00
1 Samsung Galaxy Z Flip5 
38177.05
2 Tecno Phantom V Fold 
39249.00
2 Apple i

10 realme 7 Pro 
None
10 Infinix Note 10 Pro 
10423.73
10 Infinix Note 12 G96 
7468.80
10 Tecno Camon 18P 
7499.00
10 Redmi Note 12 
8449.00
10 Samsung Galaxy A24 
8299.00
10 realme 9 Pro 
8516.00
10 Redmi Note 11S 
9999.00
10 Samsung Galaxy M23 5G 
9740.22
10 realme 8 
8633.74
10 Samsung Galaxy Note 8 
None
10 Xiaomi Redmi Note 9S 
7398.39
10 realme 6 Pro 
9500.00
10 Samsung Galaxy S9 
None
10 Redmi Note 10 
7898.23
10 Samsung Galaxy A51 
19498.00
10 Tecno Pova 4 
6449.00
10 realme 10 
8949.00
10 realme 8 Pro 
None
10 Samsung Galaxy M14 5G 
6765.25
10 Redmi Note 12 
8507.75
10 Oppo Reno4 
20000.00
11 Xiaomi Mi 9T 
None
11 Redmi Note 11 
7921.80
11 Tecno Pova 3 
13950.00
11 Redmi Note 10 
7077.51
11 Infinix Note 12 G96 
6890.00
11 realme X2 
None
11 Redmi Note 10S 
8998.00
11 Apple iPhone SE 2 
20802.00
11 TCL 10 Pro 
None
11 Infinix Note 11S 
9291.00
11 Huawei Mate 20 
None
11 Oppo Reno7 
22248.00
11 TCL 20 5G 
20999.00
11 Infinix Note 11 Pro 
8259.00
11 Tecno Camon 16 Premier 
None
1

20 Xiaomi Redmi Note 8 
None
20 Xiaomi Redmi 9 
4577.04
20 Samsung Galaxy A12 
7447.02
20 Oppo A55 
6419.00
20 HTC U Ultra 
None
20 HTC U Ultra 
None
20 Huawei Mate 9 
None
20 Samsung Galaxy M13 
5999.00
20 LG G7 Fit 
None
20 Xiaomi Mi MIX 2 
None
20 Infinix Hot 30i 
5299.00
20 Xiaomi Mi 8 SE 
None
20 OnePlus 5T 
None
20 ZTE Axon 7 
None
20 Vivo Y20s 
6599.00
20 Xiaomi Redmi Note 7 
4699.06
21 Infinix Hot 10 
4987.80
21 ZTE Blade V30 Vita 
4235.00
21 Elephone E10 Pro 
6999.00
21 Xiaomi Mi Max 3 
None
21 BlackBerry KEY2 
None
21 Omix X300 
5349.00
21 Samsung Galaxy M30 
None
21 Honor Play 
None
21 Apple iPhone 7 
None
21 Apple iPhone 7 
None
21 Asus ZenFone 5 
None
21 Samsung Galaxy A8+ Plus 
None
21 Samsung Galaxy A8+ Plus 
None
21 realme C25Y 
4278.88
21 LG V20 
None
21 Samsung Galaxy A30s 
14455.00
21 Samsung Galaxy M12 
6099.00
21 Xiaomi Mi MIX 2 Special Edition 
None
21 Oppo A5 2020 
None
21 Huawei P10 
None
21 Huawei P10 
None
21 Huawei P30 Lite 
None
21 Samsung Galaxy A21s 
7937.

30 Tecno Camon 12 Air 
None
30 LG K50s 
None
30 Huawei P Smart 2019 
3704.05
30 Meizu Pro 6 
None
30 General Mobile GM 20 
6300.00
30 General Mobile GM 20 
None
30 Casper VIA S 
None
30 Samsung Galaxy J8 
None
30 Samsung Galaxy J8 
None
30 General Mobile GM 22 
3619.00
30 Elephone PX 
None
30 Sony Xperia Z2 
None
31 Samsung Galaxy S5 
None
31 LG Q Stylus+ 
None
31 Oppo AX7 
6500.00
31 Alcatel 3 2019 
None
31 Samsung Galaxy C7 
None
31 Samsung Galaxy S5 
None
31 Casper VIA F3 
None
31 Samsung Galaxy J7 Pro 
None
31 Samsung Galaxy A6+ Plus 
None
31 LG G Flex 2 
None
31 LG K41S 
None
31 Huawei Mate S 
None
31 LG Q60 
None
31 Xiaomi Redmi 5 Plus 
None
31 Samsung Galaxy J7 Pro 
None
31 Apple iPhone 6s 
None
31 Xiaomi Mi Max 
None
31 Huawei P9 
None
31 Huawei P10 Lite 
None
31 Oppo AX7 
None
31 Asus ZenFone 2 
None
31 Sony Xperia Z3 Dual 
None
31 Sony Xperia Z3 
None
31 realme C21 
4999.00
31 Xiaomi Mi A1 
None
31 Samsung Galaxy A11 
5900.00
31 Casper VIA A3 
5240.96
31 LG G3 
None
31 Tecno 

In [3]:
df = pd.DataFrame({
    'İsim': IsimListe,
    'Fiyat' : FiyatListe,
    'Puan': PuanListe,
    'Depolama': DahiliDepolama,
    'Hat Sayısı': HatSayisi,
    'RAM': RAM,
    'Ekran Boyutu': EkranBoyutuListe,
    'Batarya Kapasitesi': BataryaKapasitesi,
    '5 G': BesG,
    '4.5 G': DortBucukG,
    'Suya Dayanıklılık': SuyaDayanaklilik,
    'Kamera Piksel' : KameraPixel,
    'Ekran Piksel' : EkranPixel,
    'Ekran/Gövde Oranı' : EkranOran,
    'CPU Frekans' : CPUFrekans,
    'CPU Cekirdek' : CPUCekirdek,
    'SAR Degeri' : SARDegeri,
    'Parmak İzi' : ParmakIzi,
    'Hızlı Şarj' : HizliSarj,
})
df

,İsim,Fiyat,Puan,Depolama,Hat Sayısı,RAM,Ekran Boyutu,Batarya Kapasitesi,5 G,4.5 G,Suya Dayanıklılık,Kamera Piksel,Ekran Piksel,Ekran/Gövde Oranı,CPU Frekans,CPU Cekirdek,SAR Degeri,Parmak İzi,Hızlı Şarj
0,Apple iPhone 15 Pro Max,96315.03,100,1,Çift Hat,8,6.7,4422,Var,Var,Var,48,1290x2796,89.1,3.64,6,0.98,Yok,Var
1,Samsung Galaxy S23 Ultra,63999.00,99,1,Çift Hat,12,6.8,5000,Var,Var,Var,200,1440x3088,89.41,3.36,8,0.963,Var,Var
2,Apple iPhone 15 Pro,81999.00,97,1,Çift Hat,8,6.1,3274,Var,Var,Var,48,1179x2556,87.35,3.64,6,0.98,Yok,Var
3,Xiaomi 13 Pro,60969.00,97,512,Çift Hat,12,6.73,4820,Var,Var,Var,50,1440x3200,89.74,3.2,8,0.998,Var,Var
4,Apple iPhone 14 Pro Max,80702.05,95,1,Çift Hat,6,6.7,4323,Var,Var,Var,48,1290x2796,87.63,3.46,6,0.99,Yok,Var
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,Honor 8A,None,27,32,Çift Hat,2,6.09,3020,Yok,Var,Yok,13,720x1560,78.82,2.3,8,0.43,Var,Yok
1205,Vodafone Smart V8,None,27,32,Tek Hat,3,5.5,3000,Yok,Var,Yok,16,1080x1920,70.34,1.4,8,0.424,Var,Var
1206,Huawei Y6s,None,27,32,Çift Hat,3,6.09,3020,Yok,Var,Yok,13,720x1560,78.82,2.3,8,NaN,Var,Yok
1207,Kaan N2,None,27,32,Tek Hat,3,5.7,3000,Yok,Var,Yok,13,720x1440,75.88,1.5,8,0.479,Var,Yok


In [4]:
df.to_excel("Verilerim.xlsx")